In [1]:
import os
import cv2
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
import os
import cv2
import numpy as np

def load_data(dataset_path, image_size=(224, 224)):
    X = []
    y = []
    dataset_path = "TB_Chest_Radiography_Database"
    label_map = {
        "normal-100":"normal",
        "tuberculosis-100":"TB"
    }

    for label_name, label_idx in label_map.items():
        class_dir = os.path.join(dataset_path, label_name)
        if not os.path.exists(class_dir):
            continue
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            try:
                image = cv2.imread(file_path)
                if image is None:
                    continue  # skip unreadable files
                image = cv2.resize(image, image_size)
                X.append(image)
                y.append(label_idx)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
                continue

    X = np.array(X)
    y = np.array(y)
    return X, y


In [3]:
# Load data
X, y = load_data("dataset_path")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Train classifier
X_train_flat = X_train.reshape((X_train.shape[0], -1))
X_test_flat = X_test.reshape((X_test.shape[0], -1))


clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_flat, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test_flat)

In [5]:

# Evaluate
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          TB       0.95      0.95      0.95        19
      normal       0.95      0.95      0.95        21

    accuracy                           0.95        40
   macro avg       0.95      0.95      0.95        40
weighted avg       0.95      0.95      0.95        40



In [6]:
# Save model
with open("tb_model.pkl", "wb") as f:
    pickle.dump(clf, f)